In [2]:
# imports
import pandas as pd
import numpy as np
import requests
import os
import json

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
# Definining API key credentials
API_KEY =os.environ.get("FOURSQUARE_API_KEY")

In [4]:
# importing result from citybikes API with all stations in Toronto
df = pd.read_csv('C:\\Users\\pruth\\Desktop\\Data Science Lectures\\Statistical-Modelling-Project\\Bike_stations.csv', delimiter=',')

In [5]:
# converting Dataframe into json
Bike_stations = df.to_json(orient='records')

In [6]:
 # Defining API endpoint
url = "https://api.foursquare.com/v3/places/search"

# headers
headers = {"Accept": "application/json",
           "Authorization": API_KEY}



In [7]:
# Bike_stations was in str datatype after converting it to json
# to iterate through it change it to list of dictionary
Bike_stations = json.loads(Bike_stations)
#type(Bike_stations)


In [ ]:
# creating empty list to store responses
response_list = []

# iteraing through each station in Bike_stations 
for station in Bike_stations:
    latitude = station.get('Latitude')
    longitude = station.get('Longitude')
    
    
     # Defining API endpoint
    url = "https://api.foursquare.com/v3/places/search"

    # headers
    headers = {"Accept": "application/json",
                 "Authorization": API_KEY}
    

    
    
    # setting up parameters for API request

    params = {
           'query': 'Indian Restaurant',
           'll': f"{latitude},{longitude}",
           'radius': 1000,
           'open_now': True,
           'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'
             }  
    
    # send the foresquare API request
    response = requests.get(url,params=params, headers=headers)
    
    
    
    # check if request was successfull (status code 200)
    if response.status_code == 200:
        #use the .json() method to parse the response content into json
        data_json = response.json()
        
        # Append the response to the list
        response_list.append(data_json)
    else:
        print(f"Request failed: {response.status_code}")

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [16]:
flattened_data = [
    {
    "Latitude": response["geocodes"]["main"].get("latitude"),
    "Longitude": response["geocodes"]["main"].get("longitude"),
    "name": response.get("name"),
    "distance": response.get("distance"),
    "address": response["location"].get("formatted_address"),
    "telephone": response.get("tel"),
    "rating": response.get("rating"),
    "open_true": response["hours"].get("open_now")
    }
  for result in response_list
  for response in result.get("results", [])
]

Put your parsed results into a DataFrame

In [19]:
indian_restaurant = pd.DataFrame(flattened_data)

In [20]:
indian_restaurant

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
0,43.672665,-79.321041,Udupi Palace,810,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",(416) 405-8189,8.6,True
1,43.672278,-79.322523,Motimahal Restaurant Ltd,798,"1422 Gerrard St E (Coxwell), Toronto ON M4L 1Z6",(416) 461-3111,7.6,True
2,43.671871,-79.324283,Bombay Chowpatty,812,"1386 Gerrard St E, Toronto ON M4L 1Z2",(416) 405-8080,7.3,True
3,43.672004,-79.322433,Regency Restaurant Inc,790,"1423 Gerrard St E (Hiawatha St), Toronto ON M4...",(416) 778-7366,6.2,True
4,43.666400,-79.316826,Butter Chicken Roti,255,"1610 Queen St E (Coxwel), Toronto ON M4L 1G2",(647) 349-7684,NaN,True
...,...,...,...,...,...,...,...,...
3267,43.641873,-79.411437,My Roti Place,982,"901 King St W (at Strachan Ave), Toronto ON M5...",(647) 295-9446,NaN,True
3268,43.638479,-79.417204,Brazen Head Irish Pub,617,"165 E. Liberty St (at Lynn Williams St.), Toro...",(416) 535-8787,6.1,True
3269,43.641873,-79.411437,My Roti Place,622,"901 King St W (at Strachan Ave), Toronto ON M5...",(647) 295-9446,NaN,True
3270,43.638768,-79.416315,Maurya East Indian Cuisine - Liberty Village,655,"150 E Liberty St, Toronto ON M6K 3R5",(647) 347-7002,NaN,True


In [21]:
# creating a csv file for Bike_stations Dataframe to use it for further steps
file_path = 'C:\\Users\\pruth\\Desktop\\Data Science Lectures\\Statistical-Modelling-Project\\indian_restaurant.csv'

# save the dataframe into CSV file path
indian_restaurant.to_csv(file_path, index=False)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:

YELP_API_KEY = os.getenv("YELP_API_KEY")

#https://api.yelp.com/v3/businesses/search

In [12]:
# creating empty list to store responses
yelp_response_list = []

# maximum number of request to send
MAX_REQUEST = 490

# initialize a request count
request_count = 0


# iteraing through each station in Bike_stations 
for station in Bike_stations:
    
    #check if the maximum number of request has been reached
    if request_count >= MAX_REQUEST:
        break
        
    latitude = station.get('Latitude')
    longitude = station.get('Longitude')
    
    
     # Defining API endpoint
    url_yelp = "https://api.yelp.com/v3/businesses/search"

    # headers
    headers = {"Accept": "application/json",
                 "Authorization": f'Bearer {YELP_API_KEY}',
                 }
    

    
    
    # setting up parameters for API request

    params = {
           'latitude': latitude,
           'longitude': longitude,
           'term': 'Indian Restaurant',
           'radius': 1000,
             }  
    
    # send the foresquare API request
    response_yelp = requests.get(url=url_yelp,params=params, headers=headers)
    
    
    # check if request was successfull (status code 200)
    if response_yelp.status_code == 200:
        #use the .json() method to parse the response content into json
        yelp_data_json = response_yelp.json()
        
        # Append the response to the list
        yelp_response_list.append(yelp_data_json)
        
        # Increment the request counter
        request_count += 1
    else:
        print(f"Request failed: {response_yelp.status_code}")

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [17]:
yelp_flattened_data = [
    {
    "name": response.get("name"),
    "distance": response.get("distance"),
    "address": response["location"].get("display_address"),
    "telephone": response.get("display_phone"),
    "rating": response.get("rating"),
    "review_counts": response.get("review_count"),
    "closed": response.get("is_closed")
    }
  for result in yelp_response_list
  for response in result.get("businesses", [])
]

Put your parsed results into a DataFrame

In [20]:
yelp_indian_restaurant = pd.DataFrame(yelp_flattened_data)

In [53]:
# creating a csv file for Bike_stations Dataframe to use it for further steps
file_path = 'C:\\Users\\pruth\\Desktop\\Data Science Lectures\\Statistical-Modelling-Project\\yelp_indian_restaurant.csv'

# save the dataframe into CSV file path
yelp_indian_restaurant.to_csv(file_path, index=False)

In [32]:
indian_restaurant = pd.read_csv('C:\\Users\\pruth\\Desktop\\Data Science Lectures\\Statistical-Modelling-Project\\indian_restaurant.csv')

# Comparing Results

We will go over indian_restaurant result that is the result we got from Foursqare API

In [22]:
indian_restaurant

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
0,43.672665,-79.321041,Udupi Palace,810,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",(416) 405-8189,8.6,True
1,43.672278,-79.322523,Motimahal Restaurant Ltd,798,"1422 Gerrard St E (Coxwell), Toronto ON M4L 1Z6",(416) 461-3111,7.6,True
2,43.671871,-79.324283,Bombay Chowpatty,812,"1386 Gerrard St E, Toronto ON M4L 1Z2",(416) 405-8080,7.3,True
3,43.672004,-79.322433,Regency Restaurant Inc,790,"1423 Gerrard St E (Hiawatha St), Toronto ON M4...",(416) 778-7366,6.2,True
4,43.666400,-79.316826,Butter Chicken Roti,255,"1610 Queen St E (Coxwel), Toronto ON M4L 1G2",(647) 349-7684,NaN,True
...,...,...,...,...,...,...,...,...
3267,43.641873,-79.411437,My Roti Place,982,"901 King St W (at Strachan Ave), Toronto ON M5...",(647) 295-9446,NaN,True
3268,43.638479,-79.417204,Brazen Head Irish Pub,617,"165 E. Liberty St (at Lynn Williams St.), Toro...",(416) 535-8787,6.1,True
3269,43.641873,-79.411437,My Roti Place,622,"901 King St W (at Strachan Ave), Toronto ON M5...",(647) 295-9446,NaN,True
3270,43.638768,-79.416315,Maurya East Indian Cuisine - Liberty Village,655,"150 E Liberty St, Toronto ON M6K 3R5",(647) 347-7002,NaN,True


In [23]:
# check any null values
indian_restaurant.isnull().sum()

Latitude        0
Longitude       0
name            0
distance        0
address         0
telephone       4
rating       1215
open_true       0
dtype: int64

In [24]:
# checking duplicating values 
indian_restaurant.duplicated().sum()

61

In [25]:
indian_restaurant.shape

(3272, 8)

We will go over indian restaurants that we got from yelp API requests

In [64]:
yelp_indian_restaurant.shape

(5120, 7)

In [65]:
# checking null values
yelp_indian_restaurant.isnull().sum()

name             0
distance         0
address          0
telephone        0
rating           0
review_counts    0
closed           0
dtype: int64

In [67]:
# checking duplicate values
yelp_indian_restaurant.duplicated().sum()

0

Which API provided you with more complete data? Provide an explanation. 

Yelp API provided more complete data as mentioned in above steps, There are no null and duplicated value in yelp result, where in foursquare there are null results in ratings and telephone numbers, also yelp provided more field atrributes from results then foursquare and yelp returns more precise and quantity result then foursqare.

One downside for yelp is that it provides less api requests then Foursqare as you have to me more carefull with code writing with yelp request as to not run out of requests

Get the top 10 restaurants according to their rating

In [69]:
# indian_restaurants foursquare result 
indian_restaurant.sort_values(by='rating',ascending=False).head(10)

,name,distance,address,telephone,rating,open_true
336,Bar Hop,431,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
1512,Bar Hop,861,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
1728,Bar Hop,623,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
591,Bar Hop,653,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
245,Bar Hop,991,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
1905,Bar Hop,292,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
745,Bar Hop,118,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
1965,Bar Hop,838,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
251,Bar Hop,315,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True
989,Bar Hop,914,"391 King St W (at Charlotte St), Toronto ON M5...",(647) 352-7476,8.7,True


In [71]:
# This is including the duplicates as well but we will keep the duplicated results for now as looking at distance it is duplicating cause the restaurants might be showing for different bike stations

In [70]:
# indian restaurant in yelp API
yelp_indian_restaurant.sort_values(by='rating', ascending=False).head(10)

,name,distance,address,telephone,rating,review_counts,closed
3995,Naans & More,437.504391,"795 Bathurst St, Toronto, ON M5S 1Z5, Canada",,5.0,1,False
617,Rikki Tikki,38.250332,"209 Augusta Avenue, Toronto, ON M5T 2L4, Canada",+1 416-792-3229,5.0,27,False
3886,Cucini Xpress,311.814521,"372 Yonge Street, Toronto, ON M5G 2K9, Canada",+1 416-827-1815,5.0,3,False
4474,Poppadum,1268.706174,"874 Yonge Street, Toronto, ON M4W 2J1, Canada",+1 647-892-2907,5.0,6,False
4475,Gully,902.299035,"521 Bloor Street W, Toronto, ON M5S 1Y4, Canada",+1 416-516-9099,5.0,1,False
380,Poppadum,1088.835400,"874 Yonge Street, Toronto, ON M4W 2J1, Canada",+1 647-892-2907,5.0,6,False
379,Gully,802.352522,"521 Bloor Street W, Toronto, ON M5S 1Y4, Canada",+1 416-516-9099,5.0,1,False
3887,Curry Kitchen + Falafel,882.480535,"139 Dundas Street E, Toronto, ON M5B 2N6, Canada",+1 647-352-8779,5.0,1,False
3243,Momo Ghar,479.511821,"568 Parliament St, Toronto, ON M4X 1P6, Canada",+1 416-920-8224,5.0,12,False
3890,Lett'us Roll,494.921617,"335 Yonge Street, World Food Market, Toronto, ...",,5.0,1,False
